# 부도기업예측을 위한 재무변수 전처리<br>

1. 거래소번호 수 6자리 맞추기 -> 0으로 맞춰도 새로 CVS read시 0 다 누락되어 처리하지않기로 함 <br>
2. nan값 해결해야합니당 -> 결측치가 많은 칼럼은 중요도를 따진 후 feature selection 전 드랍 예정<br>
3. KIND에서 가져온 공시정보다 매칭되지 않은 13개 기업 공시제목 강제할당 처리 완 


In [232]:
del train, train_full, bankrupt, merged, df

In [233]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

In [234]:
train_full = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\코드_재무변수.csv', encoding='euc-kr')
train = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\코드_재무변수.csv', encoding='euc-kr', usecols = ['회사명','거래소코드','회계년도','소속코드','상장폐지일'])
bankrupt = pd.read_csv(r'C:\Users\user\Desktop\code\부실기업\kind_bankrupt_df.csv', index_col=0)

In [235]:
#회계년도 추출
# train['년'] = train['회계년도'].str[:4]
# train_full['년'] = train_full['회계년도'].str[:4]

#회계년도+1 칼럼 생성 : 사업보고서 연도와 공시 데이터 연도 시점차이로 인해 +1
train['년'] = train['회계년도'].str[0:4].astype(int) + 1
train_full['년'] = train_full['회계년도'].str[0:4].astype(int) + 1

### KIND에서 추출한 상폐관련 공시 리스트 CSV화

In [236]:
# d0002 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0002.csv')
# d0305 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0305.csv')
# d0608 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0608.csv')
# d0911 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0911.csv')
# d1214 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1214.csv')
# d1517 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1517.csv')
# d1820 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1820.csv')

In [237]:
# bankrupt_df = pd.concat([d0002, d0305,d0608, d0911, d1214, d1517, d1820], axis=0).reset_index()
# bankrupt_df.drop(columns=['index'], inplace=True)

In [238]:
bankrupt_df

,번호,시간,회사명,종목코드,공시제목,제출인
0,24,2002-10-31 18:27,정산애강,22220,부도발생및은행거래정지,정산애강
1,23,2002-10-08 12:06,대호특수강,21040,은행거래재개,대호특수강
2,22,2002-05-21 18:16,에이치엘비,28300,회사정리절차종결결정,에이치엘비
3,21,2002-04-17 17:42,에이치엘비,28300,회사정리절차종결신청,에이치엘비
4,20,2001-12-26 14:58,대호특수강,21040,회사정리절차개시결정,대호특수강
...,...,...,...,...,...,...
389,155,2020-02-12 16:21,에스앤더블류,103230,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에스앤더블류
390,154,2020-02-11 17:02,파나진,46210,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,파나진
391,153,2020-02-10 16:32,국순당,43650,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],국순당
392,152,2020-02-10 16:24,유아이디,69330,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,유아이디


In [239]:
#저장
# bankrupt_df.to_csv('kind_bankrupt_df.csv')

### bankrupt_df 파일 불러온 후 시간 슬라이싱

In [240]:
# bankrupt = pd.read_csv(r'C:\Users\user\Desktop\code\부실기업\kind_bankrupt_df.csv', index_col=0)

In [241]:
#년도 추출
bankrupt['년'] = bankrupt['시간'].str[:4]

In [242]:
bankrupt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 393
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시간      394 non-null    object
 1   회사명     394 non-null    object
 2   종목코드    394 non-null    int64 
 3   공시제목    394 non-null    object
 4   제출인     386 non-null    object
 5   년       394 non-null    object
dtypes: int64(1), object(5)
memory usage: 21.5+ KB


In [243]:
#dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'

In [244]:
print(train.shape)
print(bankrupt.shape)

(27166, 6)
(394, 6)


### train, bankrupt merge위한 key 만들기 : 종목코드 + 년 

In [245]:
bankrupt['key'] = bankrupt['종목코드'].astype(str) +  bankrupt['년']

In [246]:
train['key'] = train['거래소코드'].astype(str) + train['년'].astype(str)
train_full['key'] = train_full['거래소코드'].astype(str) + train_full['년'].astype(str)

In [247]:
merged = pd.merge(train_full, bankrupt, on = 'key', how = 'outer')
# merged = pd.merge(train_full, bankrupt, on = 'key', how = 'outer')

In [248]:
merged

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,2001.0,588202001,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,2002.0,588202002,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,2003.0,588202003,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,2004.0,588202004,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,2005.0,588202005,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1019702015,2015-03-02 18:15,우양에이치씨,101970.0,회생절차개시신청,우양에이치씨,2015
27375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,336002020,2020-11-17 16:01,럭슬,33600.0,[정정]회생절차개시신청,럭슬,2020
27376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,336002020,2020-09-02 15:40,럭슬,33600.0,회생절차개시신청[정],럭슬,2020
27377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,862502020,2020-07-28 14:54,이노와이즈,86250.0,회생절차개시결정,화신테크,2020


In [249]:
merged[merged['회사명_x'].isna()]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-12-20 16:17,ST&I,31800.0,회사정리절차종결결정,에스티앤아이,2000
27358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-11-23 10:19,ST&I,31800.0,회사정리절차종결신청,에스티앤아이,2000
27359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (1)),NaN,2000
27360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (2)),NaN,2000
27361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,338502000,2000-08-07 15:44,지노시스템,33850.0,화의채무변제완료보고서제출,지노시스템,2000
27362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,333102000,2000-06-29 10:33,엠투엔,33310.0,화의채무변제완료보고서제출,엠투엔,2000
27363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,333102000,2000-01-31 00:00,엠투엔,33310.0,은행거래재개((주)우진산전),NaN,2000
27364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,133102000,2000-03-31 00:00,아진산업,13310.0,회사정리절차개시결정(아진산업(주)),NaN,2000
27365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-03-31 00:00,에이치엘비,28300.0,은행거래재개(국제정공(주)),NaN,2000
27366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (1)),NaN,2000


In [250]:
print('train에 반영되지않은 부도회사 명 : ','\n', merged[merged['회사명_x'].isna()]['회사명_y'].unique())
print('-------------------------------------------------------------------')
print('train에 반영되지않은 부도회사 코드 : ', '\n', merged[merged['소속코드'].isna()]['종목코드'].unique())
print('-------------------------------------------------------------------')
print('train에 반영되지않은 부도회사 계 : ', merged[merged['회사명_x'].isna()]['회사명_y'].nunique())

train에 반영되지않은 부도회사 명 :  
 ['ST&I' '지노시스템' '엠투엔' '아진산업' '에이치엘비' '엔케이바이오' 'GK파워' '디보스' '평안물산'
 '에이프로테크놀로지' '우양에이치씨' '럭슬' '이노와이즈']
-------------------------------------------------------------------
train에 반영되지않은 부도회사 코드 :  
 [ 31800.  33850.  33310.  13310.  28300.  19260.  54020.  80140.  37240.
  45470. 101970.  33600.  86250.]
-------------------------------------------------------------------
train에 반영되지않은 부도회사 계 :  13


## train에 반영되지않은 부도기업 13개 공시제목 채워넣기<br>
<span style="color:yellow">처리한 내역</span><br>
-31800.0 	(주)에스티앤아이<br>
-33850.0    (주)지노시스템<br>
-54020.0    (주)지케이파워<br>
-37240.0    평안물산(주)<br>
-45470.0    (주)에이프로테크놀로지<br>
-101970.0   우양에이치씨(주)<br>
-33600.0    럭슬(주)<br>
-86250.0    (주)화신테크<br>
<br>
-28300.0    에이치엘비(주)	<br>
-19260.0    (주)엔케이바이오<br>
-80140.0    (주)디보스<br>

<span style="color:red">**특이사항**</span><br>
-33310.0    (주)엠투엔 00년에 채무변제완료 후 은행거래 재개 -> 부도기업 아님으로 판단<br>
-13310.0    아진산업(주) -> 회사정리절차개시결정 후 20년간 영업중. 부도기업인지 아닌지 고려해봐야함

### st&i 31800.0

In [251]:
merged.loc[merged['거래소코드']==31800.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
9016,(주)에스티앤아이,31800.0,2000/12,4.0,2009/07/01,UQ,39665.0,3256.0,2928.0,511.0,...,NaN,NaN,2001.0,318002001,NaN,NaN,NaN,NaN,NaN,NaN
9017,(주)에스티앤아이,31800.0,2001/12,4.0,2009/07/01,UQ,34205.0,2258.0,1407.0,61.0,...,NaN,NaN,2002.0,318002002,NaN,NaN,NaN,NaN,NaN,NaN
9018,(주)에스티앤아이,31800.0,2002/12,4.0,2009/07/01,UQ,67242.0,37320.0,12111.0,1054.0,...,NaN,NaN,2003.0,318002003,NaN,NaN,NaN,NaN,NaN,NaN
9019,(주)에스티앤아이,31800.0,2003/12,4.0,2009/07/01,UQ,51446.0,17927.0,10882.0,2569.0,...,NaN,NaN,2004.0,318002004,NaN,NaN,NaN,NaN,NaN,NaN
9020,(주)에스티앤아이,31800.0,2004/12,4.0,2009/07/01,UQ,42000.0,11661.0,10886.0,9555.0,...,NaN,NaN,2005.0,318002005,NaN,NaN,NaN,NaN,NaN,NaN
9021,(주)에스티앤아이,31800.0,2005/12,4.0,2009/07/01,UQ,34047.0,6314.0,4646.0,150.0,...,NaN,NaN,2006.0,318002006,NaN,NaN,NaN,NaN,NaN,NaN
9022,(주)에스티앤아이,31800.0,2006/12,4.0,2009/07/01,UQ,57281.0,16155.0,13271.0,1507.0,...,NaN,NaN,2007.0,318002007,NaN,NaN,NaN,NaN,NaN,NaN
9023,(주)에스티앤아이,31800.0,2007/12,4.0,2009/07/01,UQ,65077.0,26264.0,24626.0,204.0,...,-75.77,0.0,2008.0,318002008,NaN,NaN,NaN,NaN,NaN,NaN
9024,(주)에스티앤아이,31800.0,2008/12,4.0,2009/07/01,UQ,8326.0,1243.0,1243.0,36.0,...,-116.99,NaN,2009.0,318002009,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
merged.loc[merged['종목코드']==31800.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-12-20 16:17,ST&I,31800.0,회사정리절차종결결정,에스티앤아이,2000
27358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-11-23 10:19,ST&I,31800.0,회사정리절차종결신청,에스티앤아이,2000
27359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (1)),NaN,2000
27360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (2)),NaN,2000


In [253]:
merged.loc[merged['key']=='318002000', '공시제목'] = '회사정리절차종결결정'

### **엠투엔** 33310.0 -> 부도기업 아닌걸로 판단 

In [254]:
merged.loc[merged['거래소코드']==33310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
10414,(주)엠투엔,33310.0,2000/12,5.0,NaN,QG,21993.0,9456.0,5107.0,4.0,...,NaN,NaN,2001.0,333102001,NaN,NaN,NaN,NaN,NaN,NaN
10415,(주)엠투엔,33310.0,2001/12,5.0,NaN,UQ,23635.0,9988.0,5819.0,681.0,...,NaN,NaN,2002.0,333102002,NaN,NaN,NaN,NaN,NaN,NaN
10416,(주)엠투엔,33310.0,2002/12,5.0,NaN,UQ,29815.0,15254.0,8024.0,383.0,...,NaN,NaN,2003.0,333102003,NaN,NaN,NaN,NaN,NaN,NaN
10417,(주)엠투엔,33310.0,2003/12,5.0,NaN,UQ,23895.0,14267.0,6723.0,378.0,...,NaN,NaN,2004.0,333102004,NaN,NaN,NaN,NaN,NaN,NaN
10418,(주)엠투엔,33310.0,2004/12,5.0,NaN,UQ,24814.0,14154.0,6537.0,1071.0,...,NaN,NaN,2005.0,333102005,NaN,NaN,NaN,NaN,NaN,NaN
10419,(주)엠투엔,33310.0,2005/12,5.0,NaN,UQ,25800.0,14601.0,7325.0,997.0,...,NaN,NaN,2006.0,333102006,NaN,NaN,NaN,NaN,NaN,NaN
10420,(주)엠투엔,33310.0,2006/12,5.0,NaN,UQ,28862.0,17819.0,9946.0,1343.0,...,NaN,NaN,2007.0,333102007,NaN,NaN,NaN,NaN,NaN,NaN
10421,(주)엠투엔,33310.0,2007/12,5.0,NaN,UQ,61285.0,18852.0,13862.0,3770.0,...,115.14,86.41,2008.0,333102008,NaN,NaN,NaN,NaN,NaN,NaN
10422,(주)엠투엔,33310.0,2008/12,5.0,NaN,UQ,73832.0,24728.0,10373.0,91.0,...,224.87,9.46,2009.0,333102009,NaN,NaN,NaN,NaN,NaN,NaN
10423,(주)엠투엔,33310.0,2009/12,5.0,NaN,UQ,67139.0,21664.0,12380.0,2726.0,...,82.27,41.78,2010.0,333102010,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
merged[merged['종목코드']==33310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,333102000,2000-06-29 10:33,엠투엔,33310.0,화의채무변제완료보고서제출,엠투엔,2000
27363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,333102000,2000-01-31 00:00,엠투엔,33310.0,은행거래재개((주)우진산전),NaN,2000


In [256]:
merged.loc[merged['key'] == '333102021', '공시제목'] = '화의채무변제완료보고서제출'

### 지노 33850.0 화의채무변제완료보고서제출

In [257]:
merged.loc[merged['거래소코드'] == 33850.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
14214,(주)지노시스템,33850.0,2000/12,4.0,2011/05/07,QG,13655.0,8846.0,7485.0,1919.0,...,NaN,NaN,2001.0,338502001,NaN,NaN,NaN,NaN,NaN,NaN
14215,(주)지노시스템,33850.0,2001/12,4.0,2011/05/07,UQ,22179.0,15593.0,14762.0,2219.0,...,NaN,NaN,2002.0,338502002,NaN,NaN,NaN,NaN,NaN,NaN
14216,(주)지노시스템,33850.0,2002/12,4.0,2011/05/07,UQ,25466.0,17229.0,15134.0,2891.0,...,NaN,NaN,2003.0,338502003,NaN,NaN,NaN,NaN,NaN,NaN
14217,(주)지노시스템,33850.0,2003/12,4.0,2011/05/07,UQ,26862.0,17129.0,16437.0,1644.0,...,NaN,NaN,2004.0,338502004,NaN,NaN,NaN,NaN,NaN,NaN
14218,(주)지노시스템,33850.0,2004/12,4.0,2011/05/07,UQ,16319.0,8982.0,8494.0,41.0,...,NaN,NaN,2005.0,338502005,NaN,NaN,NaN,NaN,NaN,NaN
14219,(주)지노시스템,33850.0,2005/12,4.0,2011/05/07,UQ,19363.0,7509.0,7218.0,174.0,...,NaN,NaN,2006.0,338502006,NaN,NaN,NaN,NaN,NaN,NaN
14220,(주)지노시스템,33850.0,2006/12,4.0,2011/05/07,UQ,22226.0,13669.0,13502.0,5023.0,...,NaN,NaN,2007.0,338502007,NaN,NaN,NaN,NaN,NaN,NaN
14221,(주)지노시스템,33850.0,2007/12,4.0,2011/05/07,UQ,30430.0,17580.0,17538.0,4681.0,...,-42.44,0.0,2008.0,338502008,NaN,NaN,NaN,NaN,NaN,NaN
14222,(주)지노시스템,33850.0,2008/12,4.0,2011/05/07,UQ,26845.0,12301.0,10110.0,2261.0,...,-49.88,NaN,2009.0,338502009,NaN,NaN,NaN,NaN,NaN,NaN
14223,(주)지노시스템,33850.0,2009/12,4.0,2011/05/07,UQ,20577.0,7062.0,6347.0,3359.0,...,-69.36,NaN,2010.0,338502010,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
merged.loc[merged['종목코드'] == 33850.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,338502000,2000-08-07 15:44,지노시스템,33850.0,화의채무변제완료보고서제출,지노시스템,2000


In [259]:
merged.loc[merged['key'] == '338502011', '공시제목'] = '화의채무변제완료보고서제출'

### **아진산업** 13310.0 -다시 보기 NaN친구 -> 회사정리절차개시결정 후 20년간 영업중 고려해봐야함

In [260]:
merged.loc[merged['거래소코드'] == 13310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
22951,아진산업(주),13310.0,2002/12,5.0,NaN,UQ,22950.0,8013.0,7325.0,656.0,...,NaN,NaN,2003.0,133102003,NaN,NaN,NaN,NaN,NaN,NaN
22952,아진산업(주),13310.0,2003/12,5.0,NaN,UQ,21161.0,7393.0,6661.0,9.0,...,NaN,NaN,2004.0,133102004,NaN,NaN,NaN,NaN,NaN,NaN
22953,아진산업(주),13310.0,2004/12,5.0,NaN,UQ,20043.0,4902.0,4035.0,1665.0,...,NaN,NaN,2005.0,133102005,NaN,NaN,NaN,NaN,NaN,NaN
22954,아진산업(주),13310.0,2005/12,5.0,NaN,UQ,34809.0,11729.0,10733.0,6520.0,...,NaN,NaN,2006.0,133102006,NaN,NaN,NaN,NaN,NaN,NaN
22955,아진산업(주),13310.0,2006/12,5.0,NaN,UQ,37867.0,6117.0,4882.0,1.0,...,NaN,NaN,2007.0,133102007,NaN,NaN,NaN,NaN,NaN,NaN
22956,아진산업(주),13310.0,2007/12,5.0,NaN,UQ,45183.0,8901.0,7359.0,466.0,...,12002.04,1.45,2008.0,133102008,NaN,NaN,NaN,NaN,NaN,NaN
22957,아진산업(주),13310.0,2008/12,5.0,NaN,UQ,91325.0,17752.0,15222.0,272.0,...,11623.02,3.74,2009.0,133102009,NaN,NaN,NaN,NaN,NaN,NaN
22958,아진산업(주),13310.0,2009/12,5.0,NaN,UQ,110853.0,32332.0,29509.0,246.0,...,21307.55,3.48,2010.0,133102010,NaN,NaN,NaN,NaN,NaN,NaN
22959,아진산업(주),13310.0,2010/12,5.0,NaN,UQ,150757.0,54306.0,46940.0,5377.0,...,23644.77,2.74,2011.0,133102011,NaN,NaN,NaN,NaN,NaN,NaN
22960,아진산업(주),13310.0,2011/12,5.0,NaN,UQ,187676.0,66664.0,64329.0,434.0,...,24032.74,2.20,2012.0,133102012,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
merged.loc[merged['종목코드'] == 13310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,133102000,2000-03-31 00:00,아진산업,13310.0,회사정리절차개시결정(아진산업(주)),NaN,2000


In [262]:
merged.loc[merged['key'] == '133102021', '공시제목'] = '회사정리절차개시결정(아진산업(주))'

### **에이치엘비** 28300.0 2002 회사정리절차종결 BUT 2020까지 회사운영중

In [263]:
merged.loc[merged['거래소코드'] == 28300.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
23576,에이치엘비(주),28300.0,2000/12,5.0,NaN,UQ,22460.0,3362.0,1775.0,216.0,...,NaN,NaN,2001.0,283002001,NaN,NaN,NaN,NaN,NaN,NaN
23577,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-05-21 18:16,에이치엘비,28300.0,회사정리절차종결결정,에이치엘비,2002
23578,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-04-17 17:42,에이치엘비,28300.0,회사정리절차종결신청,에이치엘비,2002
23579,에이치엘비(주),28300.0,2002/12,5.0,NaN,UQ,16407.0,2497.0,1990.0,30.0,...,NaN,NaN,2003.0,283002003,NaN,NaN,NaN,NaN,NaN,NaN
23580,에이치엘비(주),28300.0,2003/12,5.0,NaN,QG,11762.0,726.0,645.0,303.0,...,NaN,NaN,2004.0,283002004,NaN,NaN,NaN,NaN,NaN,NaN
23581,에이치엘비(주),28300.0,2004/12,5.0,NaN,UQ,2347.0,1768.0,1768.0,15.0,...,NaN,NaN,2005.0,283002005,NaN,NaN,NaN,NaN,NaN,NaN
23582,에이치엘비(주),28300.0,2005/12,5.0,NaN,UQ,19044.0,7618.0,7318.0,3858.0,...,NaN,NaN,2006.0,283002006,NaN,NaN,NaN,NaN,NaN,NaN
23583,에이치엘비(주),28300.0,2006/12,5.0,NaN,UQ,27051.0,15420.0,15063.0,7998.0,...,NaN,NaN,2007.0,283002007,NaN,NaN,NaN,NaN,NaN,NaN
23584,에이치엘비(주),28300.0,2007/12,5.0,NaN,UQ,43462.0,27231.0,27209.0,13265.0,...,-435.79,0.00,2008.0,283002008,NaN,NaN,NaN,NaN,NaN,NaN
23585,에이치엘비(주),28300.0,2008/12,5.0,NaN,UQ,50147.0,10165.0,9613.0,169.0,...,-124.04,NaN,2009.0,283002009,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
merged.loc[merged['종목코드'] == 28300.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
23577,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-05-21 18:16,에이치엘비,28300.0,회사정리절차종결결정,에이치엘비,2002
23578,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-04-17 17:42,에이치엘비,28300.0,회사정리절차종결신청,에이치엘비,2002
27365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-03-31 00:00,에이치엘비,28300.0,은행거래재개(국제정공(주)),NaN,2000
27366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (1)),NaN,2000
27367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (2)),NaN,2000


In [265]:
merged[merged['종목코드']==28300.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
23577,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-05-21 18:16,에이치엘비,28300.0,회사정리절차종결결정,에이치엘비,2002
23578,에이치엘비(주),28300.0,2001/12,5.0,NaN,UQ,17974.0,2380.0,1243.0,3.0,...,NaN,NaN,2002.0,283002002,2002-04-17 17:42,에이치엘비,28300.0,회사정리절차종결신청,에이치엘비,2002
27365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-03-31 00:00,에이치엘비,28300.0,은행거래재개(국제정공(주)),NaN,2000
27366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (1)),NaN,2000
27367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (2)),NaN,2000


### **엔케이바이오** 19260.0 ->채무변제완료 및 은행거래 재개

In [266]:
merged.loc[merged['거래소코드'] == 19260.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
10017,(주)엔케이바이오,19260.0,2000/12,4.0,2012/09/05,UQ,31785.0,7360.0,5442.0,540.0,...,NaN,NaN,2001.0,192602001,2001-02-22 18:34,엔케이바이오,19260.0,화의채무변제완료보고서제출,셀텍,2001
10018,(주)엔케이바이오,19260.0,2001/12,4.0,2012/09/05,UQ,26915.0,5537.0,4495.0,719.0,...,NaN,NaN,2002.0,192602002,NaN,NaN,NaN,NaN,NaN,NaN
10019,(주)엔케이바이오,19260.0,2002/12,4.0,2012/09/05,UQ,24184.0,4948.0,3687.0,464.0,...,NaN,NaN,2003.0,192602003,NaN,NaN,NaN,NaN,NaN,NaN
10020,(주)엔케이바이오,19260.0,2003/12,4.0,2012/09/05,UQ,23685.0,7548.0,6099.0,1321.0,...,NaN,NaN,2004.0,192602004,NaN,NaN,NaN,NaN,NaN,NaN
10021,(주)엔케이바이오,19260.0,2004/12,4.0,2012/09/05,UQ,37651.0,10036.0,8051.0,1704.0,...,NaN,NaN,2005.0,192602005,NaN,NaN,NaN,NaN,NaN,NaN
10022,(주)엔케이바이오,19260.0,2005/12,4.0,2012/09/05,UQ,43491.0,12572.0,10307.0,3095.0,...,NaN,NaN,2006.0,192602006,NaN,NaN,NaN,NaN,NaN,NaN
10023,(주)엔케이바이오,19260.0,2006/12,4.0,2012/09/05,UQ,54911.0,20037.0,18054.0,7822.0,...,NaN,NaN,2007.0,192602007,NaN,NaN,NaN,NaN,NaN,NaN
10024,(주)엔케이바이오,19260.0,2007/12,4.0,2012/09/05,UQ,40157.0,10451.0,8221.0,1348.0,...,-314.91,0.00,2008.0,192602008,NaN,NaN,NaN,NaN,NaN,NaN
10025,(주)엔케이바이오,19260.0,2008/12,4.0,2012/09/05,UQ,48850.0,6408.0,6017.0,1430.0,...,-230.81,0.00,2009.0,192602009,NaN,NaN,NaN,NaN,NaN,NaN
10026,(주)엔케이바이오,19260.0,2009/12,4.0,2012/09/05,UQ,46809.0,6424.0,5890.0,892.0,...,-127.80,0.00,2010.0,192602010,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
merged.loc[merged['종목코드'] == 19260.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
10017,(주)엔케이바이오,19260.0,2000/12,4.0,2012/09/05,UQ,31785.0,7360.0,5442.0,540.0,...,NaN,NaN,2001.0,192602001,2001-02-22 18:34,엔케이바이오,19260.0,화의채무변제완료보고서제출,셀텍,2001
27368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,192602000,2000-03-23 00:00,엔케이바이오,19260.0,은행거래재개((주)한올),NaN,2000


In [268]:
merged.loc[merged['key'] == 19260.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y


### GK파워 54020.0

In [269]:
merged.loc[merged['거래소코드'] == 54020.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
14469,(주)지케이파워,54020.0,2001/12,4.0,2010/04/03,UQ,23774.0,15963.0,9485.0,2031.0,...,NaN,NaN,2002.0,540202002,NaN,NaN,NaN,NaN,NaN,NaN
14470,(주)지케이파워,54020.0,2002/12,4.0,2010/04/03,UQ,32779.0,14046.0,8254.0,3281.0,...,NaN,NaN,2003.0,540202003,NaN,NaN,NaN,NaN,NaN,NaN
14471,(주)지케이파워,54020.0,2003/12,4.0,2010/04/03,UQ,21193.0,8242.0,5079.0,1966.0,...,NaN,NaN,2004.0,540202004,NaN,NaN,NaN,NaN,NaN,NaN
14472,(주)지케이파워,54020.0,2004/12,4.0,2010/04/03,UQ,12669.0,4478.0,3210.0,177.0,...,NaN,NaN,2005.0,540202005,NaN,NaN,NaN,NaN,NaN,NaN
14473,(주)지케이파워,54020.0,2005/12,4.0,2010/04/03,UQ,10098.0,4202.0,2169.0,758.0,...,NaN,NaN,2006.0,540202006,NaN,NaN,NaN,NaN,NaN,NaN
14474,(주)지케이파워,54020.0,2006/12,4.0,2010/04/03,UQ,9923.0,3973.0,2704.0,507.0,...,NaN,NaN,2007.0,540202007,NaN,NaN,NaN,NaN,NaN,NaN
14475,(주)지케이파워,54020.0,2007/12,4.0,2010/04/03,UQ,8611.0,3311.0,2994.0,127.0,...,-257.82,0.0,2008.0,540202008,NaN,NaN,NaN,NaN,NaN,NaN
14476,(주)지케이파워,54020.0,2008/12,4.0,2010/04/03,UQ,23590.0,19464.0,19121.0,17.0,...,-190.35,NaN,2009.0,540202009,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
merged.loc[merged['종목코드'] == 54020.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,540202010,2010-03-23 14:29,GK파워,54020.0,부도발생,지케이파워,2010


In [271]:
merged.loc[merged['key'] == '540202009', '공시제목'] = '부도발생'

### 디보스 80140.0

In [272]:
merged.loc[merged['거래소코드'] == 80140.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
2242,(주)디보스,80140.0,2002/12,4.0,2010/02/02,UQ,12254.0,7505.0,3407.0,1699.0,...,NaN,NaN,2003.0,801402003,NaN,NaN,NaN,NaN,NaN,NaN
2243,(주)디보스,80140.0,2003/12,4.0,2010/02/02,UQ,22113.0,14942.0,8272.0,544.0,...,NaN,NaN,2004.0,801402004,NaN,NaN,NaN,NaN,NaN,NaN
2244,(주)디보스,80140.0,2004/12,4.0,2010/02/02,UQ,39390.0,28852.0,21590.0,277.0,...,NaN,NaN,2005.0,801402005,NaN,NaN,NaN,NaN,NaN,NaN
2245,(주)디보스,80140.0,2005/12,4.0,2010/02/02,UQ,61574.0,40417.0,20690.0,78.0,...,NaN,NaN,2006.0,801402006,NaN,NaN,NaN,NaN,NaN,NaN
2246,(주)디보스,80140.0,2006/12,4.0,2010/02/02,UQ,74910.0,49939.0,27833.0,3809.0,...,NaN,NaN,2007.0,801402007,NaN,NaN,NaN,NaN,NaN,NaN
2247,(주)디보스,80140.0,2007/12,4.0,2010/02/02,UQ,85008.0,56026.0,34775.0,7846.0,...,-136.05,0.0,2008.0,801402008,NaN,NaN,NaN,NaN,NaN,NaN
2248,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-08-11 17:24,디보스,80140.0,[정정]회생절차개시결정,디보스,2009
2249,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-07-21 16:44,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2250,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-06-17 15:12,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2251,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-04-21 15:07,디보스,80140.0,회생절차개시결정[정],디보스,2009


In [273]:
merged.loc[merged['종목코드'] == 80140.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
2248,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-08-11 17:24,디보스,80140.0,[정정]회생절차개시결정,디보스,2009
2249,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-07-21 16:44,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2250,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-06-17 15:12,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2251,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-04-21 15:07,디보스,80140.0,회생절차개시결정[정],디보스,2009
2252,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,-321.99,NaN,2009.0,801402009,2009-04-07 17:55,디보스,80140.0,회생절차개시신청,디보스,2009
27370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,801402010,2010-01-26 14:34,디보스,80140.0,회생절차 폐지결정,디보스,2010


In [274]:
merged.loc[merged['key'] == '801402009', '공시제목']='회생절차 폐지결정'

### 평안물산 37240.0

In [275]:
merged.loc[merged['거래소코드'] == 37240.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
26333,평안물산(주),37240.0,2000/12,4.0,2012/06/01,UQ,24759.0,17714.0,15759.0,3316.0,...,NaN,NaN,2001.0,372402001,NaN,NaN,NaN,NaN,NaN,NaN
26334,평안물산(주),37240.0,2001/12,4.0,2012/06/01,UQ,48056.0,32116.0,28217.0,11055.0,...,NaN,NaN,2002.0,372402002,NaN,NaN,NaN,NaN,NaN,NaN
26335,평안물산(주),37240.0,2002/12,4.0,2012/06/01,UQ,44564.0,27632.0,24048.0,5945.0,...,NaN,NaN,2003.0,372402003,NaN,NaN,NaN,NaN,NaN,NaN
26336,평안물산(주),37240.0,2003/12,4.0,2012/06/01,UQ,32601.0,17553.0,14699.0,3084.0,...,NaN,NaN,2004.0,372402004,NaN,NaN,NaN,NaN,NaN,NaN
26337,평안물산(주),37240.0,2004/12,4.0,2012/06/01,UQ,24300.0,13351.0,9905.0,1712.0,...,NaN,NaN,2005.0,372402005,NaN,NaN,NaN,NaN,NaN,NaN
26338,평안물산(주),37240.0,2005/12,4.0,2012/06/01,UQ,17459.0,10569.0,8315.0,1475.0,...,NaN,NaN,2006.0,372402006,NaN,NaN,NaN,NaN,NaN,NaN
26339,평안물산(주),37240.0,2006/12,4.0,2012/06/01,UQ,27098.0,16208.0,15432.0,10401.0,...,NaN,NaN,2007.0,372402007,NaN,NaN,NaN,NaN,NaN,NaN
26340,평안물산(주),37240.0,2007/12,4.0,2012/06/01,UQ,48249.0,13399.0,12769.0,1745.0,...,-217.00,0.0,2008.0,372402008,NaN,NaN,NaN,NaN,NaN,NaN
26341,평안물산(주),37240.0,2008/12,4.0,2012/06/01,UQ,38425.0,8981.0,8524.0,1179.0,...,-347.31,NaN,2009.0,372402009,NaN,NaN,NaN,NaN,NaN,NaN
26342,평안물산(주),37240.0,2009/12,4.0,2012/06/01,UQ,21776.0,6177.0,6082.0,1967.0,...,-62.87,NaN,2010.0,372402010,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
merged.loc[merged['종목코드'] == 37240.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,372402012,2012-03-15 19:32,평안물산,37240.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,평안물산,2012


In [277]:
merged.loc[merged['key'] == '372402011', '공시제목'] = '내부결산시점 관리종목 지정 또는 상장폐지 사유 발생'

### 에이프로테크놀로지 45470.0

In [278]:
merged.loc[merged['거래소코드'] == 45470.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
9547,(주)에이프로테크놀로지,45470.0,2000/12,4.0,2012/04/24,UQ,8491.0,5988.0,5473.0,116.0,...,NaN,NaN,2001.0,454702001,NaN,NaN,NaN,NaN,NaN,NaN
9548,(주)에이프로테크놀로지,45470.0,2001/12,4.0,2012/04/24,UQ,16676.0,11116.0,10570.0,822.0,...,NaN,NaN,2002.0,454702002,NaN,NaN,NaN,NaN,NaN,NaN
9549,(주)에이프로테크놀로지,45470.0,2002/12,4.0,2012/04/24,UQ,15814.0,11017.0,10270.0,226.0,...,NaN,NaN,2003.0,454702003,NaN,NaN,NaN,NaN,NaN,NaN
9550,(주)에이프로테크놀로지,45470.0,2003/12,4.0,2012/04/24,UQ,17830.0,10329.0,9763.0,749.0,...,NaN,NaN,2004.0,454702004,NaN,NaN,NaN,NaN,NaN,NaN
9551,(주)에이프로테크놀로지,45470.0,2004/12,4.0,2012/04/24,UQ,19749.0,8032.0,7029.0,126.0,...,NaN,NaN,2005.0,454702005,NaN,NaN,NaN,NaN,NaN,NaN
9552,(주)에이프로테크놀로지,45470.0,2005/12,4.0,2012/04/24,UQ,25537.0,15367.0,14761.0,518.0,...,NaN,NaN,2006.0,454702006,NaN,NaN,NaN,NaN,NaN,NaN
9553,(주)에이프로테크놀로지,45470.0,2006/12,4.0,2012/04/24,UQ,56938.0,36080.0,36070.0,27.0,...,NaN,NaN,2007.0,454702007,NaN,NaN,NaN,NaN,NaN,NaN
9554,(주)에이프로테크놀로지,45470.0,2007/12,4.0,2012/04/24,UQ,18484.0,5836.0,5751.0,24.0,...,-155.28,0.00,2008.0,454702008,NaN,NaN,NaN,NaN,NaN,NaN
9555,(주)에이프로테크놀로지,45470.0,2008/12,4.0,2012/04/24,UQ,16201.0,3047.0,3001.0,545.0,...,-195.96,NaN,2009.0,454702009,NaN,NaN,NaN,NaN,NaN,NaN
9556,(주)에이프로테크놀로지,45470.0,2009/12,4.0,2012/04/24,UQ,42009.0,19975.0,14589.0,36.0,...,18.50,16.17,2010.0,454702010,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
merged.loc[merged['종목코드'] == 45470.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,454702012,2012-03-15 18:24,에이프로테크놀로지,45470.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에이프로테크놀로지,2012


In [280]:
merged.loc[merged['key'] == '454702011', '공시제목'] = '내부결산시점 관리종목 지정 또는 상장폐지 사유 발생'

### 우양에이치씨 101970.0

In [281]:
merged.loc[merged['거래소코드'] == 101970.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
24513,우양에이치씨(주),101970.0,2003/12,4.0,2015/03/17,UQ,10168.0,5048.0,4268.0,453.0,...,NaN,NaN,2004.0,1019702004,NaN,NaN,NaN,NaN,NaN,NaN
24514,우양에이치씨(주),101970.0,2004/12,4.0,2015/03/17,UQ,24334.0,7796.0,6731.0,772.0,...,NaN,NaN,2005.0,1019702005,NaN,NaN,NaN,NaN,NaN,NaN
24515,우양에이치씨(주),101970.0,2005/12,4.0,2015/03/17,UQ,34892.0,13034.0,12722.0,177.0,...,NaN,NaN,2006.0,1019702006,NaN,NaN,NaN,NaN,NaN,NaN
24516,우양에이치씨(주),101970.0,2006/12,4.0,2015/03/17,UQ,66313.0,30264.0,25145.0,143.0,...,NaN,NaN,2007.0,1019702007,NaN,NaN,NaN,NaN,NaN,NaN
24517,우양에이치씨(주),101970.0,2007/12,4.0,2015/03/17,UQ,78402.0,35089.0,31054.0,443.0,...,1576.79,2.53,2008.0,1019702008,NaN,NaN,NaN,NaN,NaN,NaN
24518,우양에이치씨(주),101970.0,2008/12,4.0,2015/03/17,UQ,131395.0,77255.0,69711.0,1422.0,...,1319.58,5.03,2009.0,1019702009,NaN,NaN,NaN,NaN,NaN,NaN
24519,우양에이치씨(주),101970.0,2009/12,4.0,2015/03/17,UQ,161973.0,100978.0,97740.0,11569.0,...,2085.08,1.95,2010.0,1019702010,NaN,NaN,NaN,NaN,NaN,NaN
24520,우양에이치씨(주),101970.0,2010/12,4.0,2015/03/17,UQ,210294.0,133253.0,125697.0,1470.0,...,1512.36,4.57,2011.0,1019702011,NaN,NaN,NaN,NaN,NaN,NaN
24521,우양에이치씨(주),101970.0,2011/12,4.0,2015/03/17,UQ,233181.0,153383.0,143876.0,4510.0,...,1611.19,3.29,2012.0,1019702012,NaN,NaN,NaN,NaN,NaN,NaN
24522,우양에이치씨(주),101970.0,2012/12,4.0,2015/03/17,UQ,316015.0,193413.0,182311.0,6934.0,...,2536.23,7.25,2013.0,1019702013,NaN,NaN,NaN,NaN,NaN,NaN


In [282]:
merged.loc[merged['종목코드'] == 101970.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1019702015,2015-03-04 16:49,우양에이치씨,101970.0,부도발생,우양에이치씨,2015
27374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1019702015,2015-03-02 18:15,우양에이치씨,101970.0,회생절차개시신청,우양에이치씨,2015


In [283]:
merged.loc[merged['key'] == '1019702014', '공시제목'] = '부도발생'

### 럭슬 33600.0

In [284]:
merged.loc[merged['거래소코드'] == 33600.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
21173,럭슬(주),33600.0,2000/12,4.0,2021/08/11,UQ,39379.0,20589.0,20383.0,4228.0,...,NaN,NaN,2001.0,336002001,NaN,NaN,NaN,NaN,NaN,NaN
21174,럭슬(주),33600.0,2001/12,4.0,2021/08/11,UQ,42842.0,18792.0,18163.0,7707.0,...,NaN,NaN,2002.0,336002002,NaN,NaN,NaN,NaN,NaN,NaN
21175,럭슬(주),33600.0,2002/12,4.0,2021/08/11,UQ,50805.0,24888.0,23638.0,7598.0,...,NaN,NaN,2003.0,336002003,NaN,NaN,NaN,NaN,NaN,NaN
21176,럭슬(주),33600.0,2003/12,4.0,2021/08/11,UQ,49951.0,20707.0,18521.0,8826.0,...,NaN,NaN,2004.0,336002004,NaN,NaN,NaN,NaN,NaN,NaN
21177,럭슬(주),33600.0,2004/12,4.0,2021/08/11,UQ,46989.0,14891.0,13739.0,5160.0,...,NaN,NaN,2005.0,336002005,NaN,NaN,NaN,NaN,NaN,NaN
21178,럭슬(주),33600.0,2005/12,4.0,2021/08/11,UQ,58238.0,16699.0,15518.0,3626.0,...,NaN,NaN,2006.0,336002006,NaN,NaN,NaN,NaN,NaN,NaN
21179,럭슬(주),33600.0,2006/12,4.0,2021/08/11,UQ,56398.0,15533.0,13996.0,3704.0,...,NaN,NaN,2007.0,336002007,NaN,NaN,NaN,NaN,NaN,NaN
21180,럭슬(주),33600.0,2007/12,4.0,2021/08/11,UQ,54904.0,18249.0,17188.0,5363.0,...,412.11,6.43,2008.0,336002008,NaN,NaN,NaN,NaN,NaN,NaN
21181,럭슬(주),33600.0,2008/12,4.0,2021/08/11,UQ,88812.0,26849.0,25896.0,6730.0,...,327.40,6.06,2009.0,336002009,NaN,NaN,NaN,NaN,NaN,NaN
21182,럭슬(주),33600.0,2009/12,4.0,2021/08/11,UQ,91332.0,27214.0,25808.0,2950.0,...,135.96,27.21,2010.0,336002010,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
merged.loc[merged['종목코드'] == 33600.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,336002020,2020-11-17 16:01,럭슬,33600.0,[정정]회생절차개시신청,럭슬,2020
27376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,336002020,2020-09-02 15:40,럭슬,33600.0,회생절차개시신청[정],럭슬,2020


In [286]:
merged.loc[merged['key'] == '336002019', '공시제목']='[정정]회생절차개시신청'

### 이노와이즈 86250.0

In [287]:
merged.loc[merged['거래소코드'] == 86250.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
19275,(주)화신테크,86250.0,2000/12,4.0,2021/08/31,UQ,15068.0,6276.0,6240.0,130.0,...,NaN,NaN,2001.0,862502001,NaN,NaN,NaN,NaN,NaN,NaN
19276,(주)화신테크,86250.0,2001/12,4.0,2021/08/31,UQ,14566.0,4417.0,4412.0,126.0,...,NaN,NaN,2002.0,862502002,NaN,NaN,NaN,NaN,NaN,NaN
19277,(주)화신테크,86250.0,2002/12,4.0,2021/08/31,UQ,13941.0,3788.0,3784.0,49.0,...,NaN,NaN,2003.0,862502003,NaN,NaN,NaN,NaN,NaN,NaN
19278,(주)화신테크,86250.0,2003/12,4.0,2021/08/31,UQ,16421.0,4539.0,4536.0,325.0,...,NaN,NaN,2004.0,862502004,NaN,NaN,NaN,NaN,NaN,NaN
19279,(주)화신테크,86250.0,2004/12,4.0,2021/08/31,UQ,19357.0,5994.0,5989.0,2179.0,...,NaN,NaN,2005.0,862502005,NaN,NaN,NaN,NaN,NaN,NaN
19280,(주)화신테크,86250.0,2005/12,4.0,2021/08/31,UQ,26217.0,7021.0,7017.0,642.0,...,NaN,NaN,2006.0,862502006,NaN,NaN,NaN,NaN,NaN,NaN
19281,(주)화신테크,86250.0,2006/12,4.0,2021/08/31,UQ,34097.0,14522.0,14518.0,1276.0,...,NaN,NaN,2007.0,862502007,NaN,NaN,NaN,NaN,NaN,NaN
19282,(주)화신테크,86250.0,2007/12,4.0,2021/08/31,UQ,33153.0,12018.0,12014.0,3118.0,...,-22.24,0.00,2008.0,862502008,NaN,NaN,NaN,NaN,NaN,NaN
19283,(주)화신테크,86250.0,2008/12,4.0,2021/08/31,UQ,35176.0,11011.0,10998.0,654.0,...,70.27,1.09,2009.0,862502009,NaN,NaN,NaN,NaN,NaN,NaN
19284,(주)화신테크,86250.0,2009/12,4.0,2021/08/31,UQ,37843.0,7507.0,7499.0,1003.0,...,116.57,14.67,2010.0,862502010,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
merged.loc[merged['종목코드'] == 86250.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,862502020,2020-07-28 14:54,이노와이즈,86250.0,회생절차개시결정,화신테크,2020
27378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,862502020,2020-05-08 15:03,이노와이즈,86250.0,회생절차개시신청,화신테크,2020


In [289]:
merged.loc[merged['key'] == '862502021', '공시제목'] = '회생절차개시신청'

In [290]:
#저장 
#merged.to_csv('재무변수_부도사유_verSH.csv')

# 하다가말았다

In [291]:
merged.head(25)

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EBITDA/평균발행주식수,EV/EBITDA,년_x,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,2001.0,588202001,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,2002.0,588202002,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,2003.0,588202003,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,2004.0,588202004,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,2005.0,588202005,NaN,NaN,NaN,NaN,NaN,NaN
5,(주)CMG제약,58820.0,2005/12,5.0,NaN,UQ,8280.0,3449.0,3226.0,609.0,...,NaN,NaN,2006.0,588202006,NaN,NaN,NaN,NaN,NaN,NaN
6,(주)CMG제약,58820.0,2006/12,5.0,NaN,UQ,10030.0,6897.0,6897.0,2850.0,...,NaN,NaN,2007.0,588202007,NaN,NaN,NaN,NaN,NaN,NaN
7,(주)CMG제약,58820.0,2007/12,5.0,NaN,UQ,22074.0,8952.0,8638.0,2714.0,...,-104.44,0.00,2008.0,588202008,NaN,NaN,NaN,NaN,NaN,NaN
8,(주)CMG제약,58820.0,2008/12,5.0,NaN,UQ,34557.0,16725.0,15015.0,6233.0,...,-221.19,NaN,2009.0,588202009,NaN,NaN,NaN,NaN,NaN,NaN
9,(주)CMG제약,58820.0,2009/12,5.0,NaN,UQ,22836.0,9827.0,7398.0,1063.0,...,-443.98,NaN,2010.0,588202010,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
merged.isna().sum()

회사명_x       22
거래소코드       22
회계년도        22
소속코드        22
상장폐지일    21943
         ...  
회사명_y    26985
종목코드     26985
공시제목     26976
제출인      26993
년_y      26985
Length: 275, dtype: int64

In [293]:
df = merged[merged['회사명_x'].notna()]

In [294]:
df.drop(columns=['시간','회사명_y','종목코드', '제출인', '년_y'],inplace=True)

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [295]:
df.rename(columns = {'회사명_x' : '회사명'}, inplace = True)
df.rename(columns = {'년_x' : '년'}, inplace = True)
df

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,PSR(Price sales ratio)(최저),기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,NaN
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,NaN
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,588202003,NaN
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,588202004,NaN
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,588202005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,NaN
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,NaN
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,NaN
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,NaN


In [296]:
df.to_csv('임시.csv')

In [297]:
df.loc[(df['소속코드'] == 4) & 
((df['감사의견코드'] == 'DS') | (df['감사의견코드'] == 'DU') | (df['감사의견코드'] == 'AG') | (df['감사의견코드'] == 'DI') | (df['감사의견코드'] == 'QS')
| (df['공시제목'].notna())), '부도'] = 0

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [298]:
df['부도']=df['부도'].fillna(1)

C:\Users\user\AppData\Local\Temp/ipykernel_6632/1167929933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['부도']=df['부도'].fillna(1)


In [299]:
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,NaN,1.0
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,NaN,1.0
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,588202003,NaN,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,588202004,NaN,1.0
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,588202005,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,NaN,1.0
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,NaN,1.0
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,NaN,1.0
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,NaN,1.0


In [300]:
df['부도'].value_counts()

1.0    26915
0.0      442
Name: 부도, dtype: int64

In [ ]:
df.to_csv('재무변수_부도사유_ver2SH.csv')

In [301]:
#df.to_csv('재무변수_부도사유_verSH1130.csv')